## 4. Transfer leaning using ResNet50 
Due to the small image size, we next tried transfer leaning using a commonly used pre-trained model ResNet50. The result also suggests overfitting of the training data and labeling all testing data into one category. This is likely due to ResNet50 was not previously trained on x-ray images, and features it has learned might be distracting. 

In [1]:
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 200
WIDTH = 200

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH,3))


Using Theano backend.
/Users/jtian/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/Users/jtian/anaconda3/lib/python3.7/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [2]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model


In [3]:
class_list = ["0", "1"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


In [4]:
import numpy as np
X_train=np.load("./X_train.npy")
Y_train=np.load("./Y_train.npy")
print(X_train.shape)

(200, 1, 200, 200)


In [5]:
X=X_train[:,0,:,:]


In [6]:
rgb_batch = np.repeat(X[..., np.newaxis], 3, -1)
print(rgb_batch.shape)

(200, 200, 200, 3)


In [7]:

from keras.optimizers import SGD, Adam
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
finetune_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [8]:
finetune_model.fit(rgb_batch, Y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
200/200 [==============================] - 82s 411ms/step - loss: 7.0374 - acc: 0.5050
Epoch 2/10
200/200 [==============================] - 81s 407ms/step - loss: 8.0590 - acc: 0.5000
Epoch 3/10
200/200 [==============================] - 82s 408ms/step - loss: 8.0590 - acc: 0.5000
Epoch 4/10
200/200 [==============================] - 83s 415ms/step - loss: 8.0590 - acc: 0.5000
Epoch 5/10
200/200 [==============================] - 88s 439ms/step - loss: 8.0590 - acc: 0.5000
Epoch 6/10
200/200 [==============================] - 83s 417ms/step - loss: 8.0590 - acc: 0.5000
Epoch 7/10
200/200 [==============================] - 83s 416ms/step - loss: 8.0590 - acc: 0.5000
Epoch 8/10
200/200 [==============================] - 84s 418ms/step - loss: 8.0590 - acc: 0.5000
Epoch 9/10
200/200 [==============================] - 84s 418ms/step - loss: 8.0590 - acc: 0.5000
Epoch 10/10
200/200 [==============================] - 84s 419ms/step - loss: 8.0590 - acc: 0.5000


In [9]:
X_test=np.load("./X_test.npy")
Y_test=np.load("./Y_test.npy")
print(X_test.shape)

(100, 1, 200, 200)


In [10]:
Xtest=X_test[:,0,:,:]
Xtest_new = np.repeat(Xtest[..., np.newaxis], 3, -1)
print(Xtest_new.shape)


(100, 200, 200, 3)


In [11]:
score = finetune_model.evaluate(Xtest_new, Y_test, verbose=0)


In [12]:
print(score)


[8.22022876739502, 0.49]


In [ ]:
# using Adam optimizer

In [15]:
class_list = ["0", "1"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


In [16]:
adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:

finetune_model.fit(rgb_batch, Y_train, batch_size=32, epochs=10, verbose=1)


Epoch 1/10
200/200 [==============================] - 86s 429ms/step - loss: 1.5421 - acc: 0.5100
Epoch 2/10
200/200 [==============================] - 84s 419ms/step - loss: 1.1365 - acc: 0.5850
Epoch 3/10
200/200 [==============================] - 87s 436ms/step - loss: 0.6412 - acc: 0.7450
Epoch 4/10
200/200 [==============================] - 87s 434ms/step - loss: 0.4914 - acc: 0.7950
Epoch 5/10
200/200 [==============================] - 85s 423ms/step - loss: 0.3864 - acc: 0.8600
Epoch 6/10
200/200 [==============================] - 94s 471ms/step - loss: 0.3197 - acc: 0.8750
Epoch 7/10
200/200 [==============================] - 84s 421ms/step - loss: 0.2717 - acc: 0.8950
Epoch 8/10
200/200 [==============================] - 83s 415ms/step - loss: 0.2389 - acc: 0.9050
Epoch 9/10
200/200 [==============================] - 83s 415ms/step - loss: 0.2517 - acc: 0.9050
Epoch 10/10
200/200 [==============================] - 84s 422ms/step - loss: 0.0984 - acc: 0.9600


In [18]:
score2 = finetune_model.evaluate(Xtest_new, Y_test, verbose=0)
print(score2)


[0.7427052509784698, 0.51]


In [24]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [25]:
class_list = ["0", "1"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


In [26]:
adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
EPOCHS = 10
BS = 32
H = finetune_model.fit_generator(aug.flow(rgb_batch[:90]+rgb_batch[-90:], Y_train[:90]+Y_train[-90:], batch_size=BS),
                                 validation_data=(rgb_batch[91:109], Y_train[91:109]), 
                                 steps_per_epoch=len(rgb_batch[:90]+rgb_batch[-90:]) // BS,epochs=EPOCHS)

Epoch 1/10
2/2 [==============================] - 31s 15s/step - loss: 3.0074 - acc: 0.4305 - val_loss: 0.7369 - val_acc: 0.5000
Epoch 2/10
2/2 [==============================] - 30s 15s/step - loss: 2.9809 - acc: 0.4749 - val_loss: 0.7026 - val_acc: 0.5000
Epoch 3/10
2/2 [==============================] - 34s 17s/step - loss: 2.8912 - acc: 0.5625 - val_loss: 0.6981 - val_acc: 0.5000
Epoch 4/10
2/2 [==============================] - 32s 16s/step - loss: 2.7417 - acc: 0.4862 - val_loss: 0.7003 - val_acc: 0.5000
Epoch 5/10
2/2 [==============================] - 33s 17s/step - loss: 2.9769 - acc: 0.4531 - val_loss: 0.6992 - val_acc: 0.5000
Epoch 6/10
2/2 [==============================] - 28s 14s/step - loss: 3.1640 - acc: 0.4467 - val_loss: 0.6968 - val_acc: 0.5000
Epoch 7/10
2/2 [==============================] - 28s 14s/step - loss: 2.8834 - acc: 0.4377 - val_loss: 0.6956 - val_acc: 0.5000
Epoch 8/10
2/2 [==============================] - 28s 14s/step - loss: 2.8975 - acc: 0.5557 - val

In [28]:
score3 = finetune_model.evaluate(Xtest_new, Y_test, verbose=0)
print(score3)


[0.6891342234611512, 0.52]
